In [1]:
import pandas as pd

# POC
Let's assume we have ran a sinlge simulation which output the following:

In [2]:
df = pd.read_csv("dummy_sim.csv", index_col=0)
df.head()

,team,finish,start,dnf,fastest_lap
driver,,,,,
Hamilton,Mercedes,1,1,0,0
Verstappen,RedBull,2,2,0,1
Leclerc,Ferrari,3,3,0,0
Bottas,Mercedes,4,20,0,0
Vettel,Ferrari,5,4,0,0


Our goal is to translate this sim result into the number of points gained per constructor and per driver

## Driver Points

### Qualy Session Points
3 points for Q3, 2 Points for Q2 and 1 Point for Q1 

In [3]:
q3 = df.eval("start <= 10").mul(3)
q2 = df.eval("10 < start <= 15").mul(2)
q1 = df.eval("15 < start").mul(1)
df["qualy_session_points"] = q1 + q2 + q3

### Qualy Position Bonuses
10 Points for pole, 9 points for 2nd, ..., 1 point for 10th

In [4]:
qualy_bonus = 11 - df["start"]
qualy_bonus[qualy_bonus < 0] = 0
df["qualy_bonus_points"] = qualy_bonus

### vs Team Mate
2 Points for qualifying and 3 points for finishing ahead of team mate

In [5]:
team_groups = df.groupby("team").groups

for team in team_groups:
    team_df = df.loc[team_groups[team]][["start", "finish"]]
    qualy_winner = team_df.sort_values("start").index[0]
    race_winner = team_df.sort_values("finish").index[0]
    df.loc[qualy_winner, "vs_qualy_points"] = 2
    df.loc[race_winner, "vs_race_points"] = 3

df = df.fillna(0)

### Finishing Race
1 Point for finishing the race

In [6]:
df["finished_race_points"] = 1 - df["dnf"]

### Points Diff
2 Points per position gained and -2 or -1 points per position lost for inside or outside of top 10.

In [7]:
positions_gained = 2 * (df["start"] - df["finish"])
positions_gained[positions_gained < 0] = 0
positions_gained[positions_gained > 10] = 10
df["positions_gained_points"] = positions_gained

In [8]:
positions_lost_top_10 = -2 * (df["finish"] - df["start"])
positions_lost_top_10[positions_lost_top_10 > 0] = 0
positions_lost_top_10[df["start"] > 10] = 0
positions_lost_top_10[positions_lost_top_10 < -10] = -10

positions_lost_bottom_10 = df["start"] - df["finish"]
positions_lost_bottom_10[positions_lost_bottom_10 > 0] = 0
positions_lost_bottom_10[df["start"] <= 10] = 0
positions_lost_bottom_10[positions_lost_bottom_10 < -5] = -5

positions_lost_points = positions_lost_top_10 + positions_lost_bottom_10
df["positions_lost_points"] = positions_lost_points

### Finishing Position Bonuses
Points for top 10 finishes

In [9]:
points_by_position = {
    1: 25,
    2: 18,
    3: 15,
    4: 12,
    5: 10,
    6: 8,
    7: 6,
    8: 4,
    9: 2,
    10: 1,
    0: 0,
}
finishing_position_points = df["finish"].copy()
finishing_position_points[finishing_position_points > 10] = 0
finishing_position_points = finishing_position_points.map(points_by_position)
df["finish_position_points"] = finishing_position_points

### Fastest Lap
5 Points for Fastest Lap


In [12]:
df["fastest_lap_points"] = df["fastest_lap"] * 5

# summary

For the drivers points we just sum across all columns. For constructors we sum across the following columns and sum both drivers scores:
 - qualy_session_points
 - qualy_bonus_points
 - finished_race_points
 - positions_gained_points
 - positions_lost_points
 - finishing_position_points

In [28]:
df.drop(axis=1, labels=["team", "finish", "start", "dnf"]).sum(axis=1).sort_values(ascending=False)

driver
Hamilton      44.0
Verstappen    42.0
Leclerc       32.0
Bottas        24.0
Perez         22.0
Vettel        19.0
Albon         16.0
Kvyat         16.0
Norris        14.0
Raikkonen     13.0
Russell        9.0
Magnussen      8.0
Ricciardo      5.0
Sainz          4.0
Giovinazzi     4.0
Grosjean       3.0
Latifi         2.0
Ocon           0.0
Gasly          0.0
Stroll        -3.0
dtype: float64

In [27]:
df.reset_index()[
    [
        "team",
        "qualy_session_points",
        "qualy_bonus_points",
        "finished_race_points",
        "positions_gained_points",
        "positions_lost_points",
        "finish_position_points",
    ]
].groupby("team").sum().sum(axis=1).sort_values(ascending=False)

team
Mercedes       63
RedBull        47
Ferrari        46
RacingPoint    14
McLaren        13
AlphaRomeo     12
AlphaTauri     11
Williams        6
Haas            6
Renault         0
dtype: int64